In [1]:
# installing open java development kit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# getting spark package
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz

# unzipping
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

# Installing findspark
!pip install -q findspark

In [2]:
# importing OS
import os

# setting java environment
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# setting spark environment
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [3]:
# importing spark
import findspark

# initializing spark
findspark.init()

In [4]:
# importing spark session
from pyspark.sql import SparkSession

# creating spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

# getting sparkcontext from spark
sc = spark.sparkContext

# print
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
# defining the file path
path = "/content/wildlife_trade.csv"

In [6]:
# reading the csv file as spark dataframe
df = spark.read.option("header",True).csv(path)


In [7]:
#creating a temporary sql table
df.createOrReplaceTempView("wildlife_trade")

1. what is the most frequent Class of animal traded

In [9]:
# the query is selecting class and count the frequency of each class excluding the null values from the sql table, grouping by class
# it is then ordering them based on frequencies in decending order and storing the first class in the variable
frequent_class = spark.sql("SELECT Class, COUNT(*) as Frequency " +
                           "FROM wildlife_trade " +
                           "WHERE Class IS NOT NULL " +
                           "GROUP BY Class " +
                           "ORDER BY Frequency DESC " +
                           "LIMIT 1")

In [10]:
# print the answer
frequent_class.show()

+--------+---------+
|   Class|Frequency|
+--------+---------+
|Reptilia|    18430|
+--------+---------+



2. List all the items (Term) traded that are associated with Mammals

In [11]:
# this query selects all distinct terms where the class is mammalia from the table
traded_items = spark.sql("SELECT DISTINCT Term " +
                         "FROM wildlife_trade " +
                         "WHERE class LIKE '%Mammalia%' ")

In [12]:
# print
traded_items.show()

+--------------------+
|                Term|
+--------------------+
|                gall|
|                meat|
| fur product (small)|
|              fibres|
|              scales|
|         bone pieces|
|            garments|
|            medicine|
|              bodies|
|            carvings|
|               horns|
|leather products ...|
|               claws|
|              skulls|
|             extract|
|               tails|
|                hair|
|           specimens|
|         skin pieces|
|         horn pieces|
+--------------------+
only showing top 20 rows



3. List all CITES Appendix II species

In [13]:
# selecting the distinct tarox where the appendix is 2
app_II_species = spark.sql("SELECT DISTINCT Taxon "+
                                   "FROM wildlife_trade "+
                                   "WHERE `App.` = 'II'")

In [14]:
# print
app_II_species.show()

+--------------------+
|               Taxon|
+--------------------+
|   Aquila chrysaetos|
|     Buteo polyosoma|
|Ferocactus chrysa...|
|Geohintonia mexicana|
|Melocactus broadwayi|
|        Parodia spp.|
|     Cebus albifrons|
|Zygosicyos tripar...|
|Euphorbia longifolia|
|Cyphastrea microp...|
|   Chalcopsitta atra|
|Myrmecophaga trid...|
|    Nepenthes hamata|
|  Acianthera calypso|
|  Aerides flabellata|
|Bulbophyllum elon...|
|Bulbophyllum odor...|
|Cephalantheropsis...|
|Dendrobium nathan...|
|Dendrobium tanger...|
+--------------------+
only showing top 20 rows



4. What is the most common animal (Taxon) traded in 2017?

In [15]:
# the query is selecting taxon and count the frequency of each taxon where year is 2017, grouping them by taxon
# it is then ordering them based on frequencies in decending order and storing the first class in the variable
m_animal_traded = spark.sql("SELECT Taxon, COUNT(*) as Frequency "+
                            "FROM wildlife_trade "+
                            "WHERE Year = 2017 "+
                            "GROUP BY Taxon "
                            "ORDER BY Frequency DESC "+
                            "LIMIT 1 ")

In [16]:
# print
m_animal_traded.show()

+--------------------+---------+
|               Taxon|Frequency|
+--------------------+---------+
|Alligator mississ...|       18|
+--------------------+---------+



5. List all the Classes of animals where the following items are traded

   a) teeth

   b) live

   c) carvings

In [17]:
# listing distinct class where the term is "teeth"
teeth = spark.sql("SELECT DISTINCT Class "+
                  "FROM wildlife_trade "+
                  "WHERE Term = 'teeth'")

# listing distinct class where the term is "live"
live = spark.sql("SELECT DISTINCT Class "+
                 "FROM wildlife_trade "+
                 "WHERE Term = 'live'")

# listing distinct class where the term is "carvings"
carvings = spark.sql("SELECT DISTINCT Class "+
                     "FROM wildlife_trade "+
                     "WHERE Term = 'carvings'")

In [18]:
# print

print("A, The Animal classes where teeth is traded:")
teeth.show()

print("B, The Animal classes where live is traded:")
live.show()

print("C, The Animal classes where carvings is traded:")
carvings.show()

A, The Animal classes where teeth is traded:
+--------+
|   Class|
+--------+
|Mammalia|
|Reptilia|
+--------+

B, The Animal classes where live is traded:
+--------------+
|         Class|
+--------------+
|          Aves|
|      Amphibia|
|       Insecta|
|      Mammalia|
|Elasmobranchii|
|     Arachnida|
|          null|
|    Gastropoda|
|      Reptilia|
|   Actinopteri|
|      Anthozoa|
|     Dipneusti|
|  Hirudinoidea|
|      Hydrozoa|
|      Bivalvia|
+--------------+

C, The Animal classes where carvings is traded:
+----------+
|     Class|
+----------+
|      Aves|
|  Bivalvia|
|  Mammalia|
|      null|
|Gastropoda|
|  Reptilia|
|  Anthozoa|
|  Hydrozoa|
+----------+

